In [1]:
from wrapper_functions import *

2023-03-08 16:39:38.719806: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/liyanran/opt/anaconda3/envs/BNE/lib/python3.7/site-packages/gpflow/experimental/utils.py:43: UserWarning: You're calling gpflow.experimental.check_shapes.decorator.check_shapes which is considered *experimental*. Expect: breaking changes, poor documentation, and bugs.
  f"You're calling {name} which is considered *experimental*."
/Users/liyanran/opt/anaconda3/envs/BNE/lib/python3.7/site-packages/gpflow/experimental/utils.py:43: UserWarning: You're calling gpflow.experimental.check_shapes.inheritance.inherit_check_shapes which is considered *experimental*. Expect: breaking changes, poor documentation, and bugs.
  f"You're calling {name} which i

TensorFlow version: 2.10.0. Expected: 2.7.0
TensorFlow Probability version: 0.18.0. Expected: 0.15.0


2023-03-08 16:40:07.522488: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Data Preparation and Parameters Setup

In [2]:
# use BAE as the 1st step
# Optimization configs. 
# Consider reduce below parameters / set to `False` if MCMC is taking too long:
# mcmc_num_steps, mcmc_burnin, mcmc_nchain, mcmc_initialize_from_map.
map_step_size=5e-4   # @param
map_num_steps=10_000  # @param

mcmc_step_size=1e-4 # @param
mcmc_num_steps=1000 # @param

mcmc_nchain=10 # @param
mcmc_burnin=2_500 # @param
bne_mcmc_initialize_from_map="True" # @param ["False", "True"]

bne_mcmc_initialize_from_map = eval(bne_mcmc_initialize_from_map)


# BMA parameters.
y_noise_std = 0.01  # Note: Changed from 0.1 # @param
bma_gp_lengthscale = .55 # @param
bma_gp_l2_regularizer = .3 # @param

bma_n_samples_train = 100 # @param
bma_n_samples_eval = 250 # @param
bma_n_samples_test = 250 # @param
bma_seed = 0 # @param

# ### Read training/prediction data
training_eastMA = pd.read_csv('../data/training_dataset/training_eastMA.csv')
training_eastMA_noMI = training_eastMA[:51]
training_eastMA_folds = pd.read_csv('../data/training_dataset/training_eastMA_folds.csv')
base_model_predictions_eastMA = pd.read_csv('../data/prediction_dataset/base_model_predictions_eastMA.csv')

print("pred longitude max and min", base_model_predictions_eastMA["lon"].max(),base_model_predictions_eastMA["lon"].min())
print("pred latitude max and min", base_model_predictions_eastMA["lat"].max(),base_model_predictions_eastMA["lat"].min())
#list(base_model_predictions_eastMA.columns)
print("train longitude max and min", training_eastMA["lon"].max(),training_eastMA["lon"].min())
print("train latitude max and min", training_eastMA["lat"].max(),training_eastMA["lat"].min())


training51= pd.read_csv('../data/training_dataset/training51.csv')

# standardize
X_train1 = np.asarray(training_eastMA_noMI[["lon", "lat"]].values.tolist()).astype(np.float32)
X_test1 = np.asarray(base_model_predictions_eastMA[["lon", "lat"]].values.tolist()).astype(np.float32)
X_valid = np.concatenate((X_train1, X_test1), axis=0)
X_centr = np.mean(X_valid, axis=0)
X_scale = np.max(X_valid, axis=0) - np.min(X_valid, axis=0)

X_train1 = (X_train1 - X_centr) / X_scale
X_test1 = (X_test1 - X_centr) / X_scale

Y_train = np.expand_dims(training_eastMA_noMI["aqs"], 1).astype(np.float32)
#Y_test = np.expand_dims(base_model_predictions_eastMA["pred_av"], 1).astype(np.float32)

print("2011 center and scale: ", X_centr, X_scale)


base_model_names = ["pred_av", "pred_gs", "pred_caces"]
base_preds_train = tf.stack([training_eastMA_noMI[base_model_name].astype(np.float32) for base_model_name in base_model_names], axis=-1)
base_preds_test = tf.stack([base_model_predictions_eastMA[base_model_name].astype(np.float32) for base_model_name in base_model_names], axis=-1)


# Assemble into configs.
bma_model_config = DEFAULT_GP_CONFIG.copy()
map_config = DEFAULT_MAP_CONFIG.copy()
mcmc_config = DEFAULT_MCMC_CONFIG.copy()

bma_model_config.update(dict(lengthscale=bma_gp_lengthscale,
                             l2_regularizer=bma_gp_l2_regularizer,
                             y_noise_std=y_noise_std,
                             #activation='relu',
                             activation_func='softmax'))
print(bma_model_config)

bma_config=dict(gp_lengthscale=bma_gp_lengthscale,
                gp_l2_regularizer=bma_gp_l2_regularizer,
                y_noise_std=y_noise_std,
                map_step_size=map_step_size,
                map_num_steps=map_num_steps,
                mcmc_step_size=mcmc_step_size,
                mcmc_num_steps=mcmc_num_steps,
                mcmc_initialize_from_map=False,
                n_samples_eval=bma_n_samples_eval,
                n_samples_train=bma_n_samples_train,
                n_samples_test=bma_n_samples_test,
                seed=bma_seed)
print(bma_config)



# BNE parameters.
bne_gp_lengthscale = 7.5  # 5. # @param
bne_gp_l2_regularizer = 10  # 15 # @param
bne_variance_prior_mean = -2.5  # @param
bne_skewness_prior_mean = -2.5  # @param
bne_seed = 0  # @param
bne_config = dict(gp_lengthscale=bne_gp_lengthscale,
                  gp_l2_regularizer=bne_gp_l2_regularizer,
                  variance_prior_mean=bne_variance_prior_mean,
                  skewness_prior_mean=bne_skewness_prior_mean,
                  map_step_size=map_step_size,
                  map_num_steps=map_num_steps,
                  mcmc_step_size=mcmc_step_size,
                  mcmc_num_steps=mcmc_num_steps,
                  mcmc_nchain=mcmc_nchain,
                  mcmc_burnin=mcmc_burnin,
                  mcmc_initialize_from_map=bne_mcmc_initialize_from_map,
                  seed=bne_seed)

print(bne_config)

map_config.update(dict(learning_rate=map_step_size,
                       num_steps=map_num_steps))

mcmc_config.update(dict(step_size=mcmc_step_size, 
                        num_steps=mcmc_num_steps,
                       burnin=mcmc_burnin,
                       nchain=mcmc_nchain,
                       debug_mode=False))



pred longitude max and min -69.93 -73.5
pred latitude max and min 44.3 40.6
train longitude max and min -70.023598 -73.443056
train latitude max and min 44.107524 40.74529
2011 center and scale:  [-72.185104  42.680347] [3.5699997 3.7000008]
{'lengthscale': 0.55, 'l2_regularizer': 0.3, 'hidden_units': 128, 'y_noise_std': 0.01, 'activation_func': 'softmax'}
{'gp_lengthscale': 0.55, 'gp_l2_regularizer': 0.3, 'y_noise_std': 0.01, 'map_step_size': 0.0005, 'map_num_steps': 10000, 'mcmc_step_size': 0.0001, 'mcmc_num_steps': 1000, 'mcmc_initialize_from_map': False, 'n_samples_eval': 250, 'n_samples_train': 100, 'n_samples_test': 250, 'seed': 0}
{'gp_lengthscale': 7.5, 'gp_l2_regularizer': 10, 'variance_prior_mean': -2.5, 'skewness_prior_mean': -2.5, 'map_step_size': 0.0005, 'map_num_steps': 10000, 'mcmc_step_size': 0.0001, 'mcmc_num_steps': 1000, 'mcmc_nchain': 10, 'mcmc_burnin': 2500, 'mcmc_initialize_from_map': True, 'seed': 0}


In [ ]:

bae_bma_diff_line = []
bae_bma_diff_df = pd.DataFrame(columns=['lon', 'lat', 'diff'])
nll_lr_each, nll_gam_each, nll_bma_mean_each, nll_bma_each, nll_bae_each = [], [], [], [], []

fold_num = 0
kf = KFold(n_splits=10, random_state=bma_seed, shuffle=True)
# rmse_bma_mean, rmse_bma2, rmse_bae = [], [], []
for train_index, test_index in kf.split(X_train1):
    fold_num += 1
    #print("Train:", train_index, "Validation:",test_index)
    X_tr, X_te = X_train1[train_index], X_train1[test_index]
    Y_tr, Y_te = Y_train[train_index], Y_train[test_index]

    base_preds_tr, base_preds_te = base_preds_train.numpy(
    )[train_index], base_preds_train.numpy()[test_index]
    print(train_index, test_index)
    print(X_tr.shape, X_te.shape, Y_tr.shape, Y_te.shape,
          base_preds_tr.shape, base_preds_te.shape)
       
    # Create data dictionary.
    data_dicts = dict(X_train=X_tr,
                      X_test=X_te,
                      Y_train=Y_tr,
                      base_preds_train=base_preds_tr,
                      base_preds_test=base_preds_te)

    print(Y_te)
    # BMA-mean.
    print('BMA-mean:', flush=True)
    data_dict, bma_mean_joint_samples = get_bma_result(
        data_dicts, bma_config=bma_config)
    y_pred_bma_mean = np.mean(np.nan_to_num(
        bma_mean_joint_samples['y']), axis=0)
    print(y_pred_bma_mean)
    pred_var_bma_mean = calc_prediction_std(y_pred_bma_mean, Y_te)
    nll_bma_mean_each = (y_pred_bma_mean - Y_te)**2

    # BMA.
    print('BMA:', flush=True)
    bma_var_config = bne_config.copy()
    bma_var_config['mcmc_initialize_from_map'] = bma_config['mcmc_initialize_from_map']
    bma_joint_samples = get_bne_result(data_dict, moment_mode='none',
                                       bne_config=bma_var_config)
    y_pred_bma = np.mean(np.nan_to_num(bma_joint_samples['y']), axis=0)
    print(y_pred_bma)
    pred_var_bma = calc_prediction_std(y_pred_bma, Y_te)
    nll_bma_each = (y_pred_bma - Y_te)**2

    # BAE.
    print('BAE:', flush=True)
    bae_joint_samples = get_bne_result(data_dict, moment_mode='mean',
                                       bne_config=bne_config)
    y_pred_bae = np.mean(np.nan_to_num(bae_joint_samples['y']), axis=0)
    print(y_pred_bae)
    pred_var_bae = calc_prediction_std(y_pred_bae, Y_te)
    nll_bae_each = (y_pred_bae - Y_te)**2


    bae_bma_diff = pd.DataFrame(columns=['lon', 'lat', 'diff'])
    bae_bma_diff["lon"] = X_te[:, 0]
    bae_bma_diff["lat"] = X_te[:, 1]
    bae_bma_diff["diff"] = nll_bae_each - nll_bma_mean_each
    bae_bma_diff_df = bae_bma_diff_df.append(bae_bma_diff)

    print(bae_bma_diff_df)
    bae_bma_diff_line.append((pred_var_bae**2 - pred_var_bma_mean**2))
    # larger points are more error
    coordinate = np.asarray(base_model_predictions_eastMA[[
                        "lon", "lat"]].values.tolist()).astype(np.float32)

    nll_list = [nll_bma_mean_each, nll_bma_each, nll_bae_each]
    nll_list = np.asarray(nll_list)
    nll_list = nll_list.reshape(-1, len(nll_bma_mean_each)*3)

    color_norm_nll_each = make_color_norm(
        nll_list.reshape(len(nll_list), -1),method='percentile')

    # plot the bae_bma_diff_df
    plt.figure(figsize=(5, 4))
    plt.scatter(X_tr[:, 0] * X_scale[0] + X_centr[0], X_tr[:, 1] * X_scale[1] +
            X_centr[1], c="black", s=10, alpha=0.5, marker="s")  # different shape
    plt.scatter(X_te[:, 0] * X_scale[0] + X_centr[0], X_te[:, 1] * X_scale[1] + X_centr[1], c=nll_bma_mean_each,
                s=abs(nll_bma_mean_each)*200, cmap='inferno_r', norm=color_norm_nll_each)

    cbar = plt.colorbar()

    plt.title("Fold " + str(fold_num))

    plt.xlim(min(coordinate[:, 0]), max(coordinate[:, 0]))
    plt.ylim(min(coordinate[:, 1]), max(coordinate[:, 1]))
    plt.show()

    # plot the bae_bma_diff_df
    plt.figure(figsize=(5, 4))
    plt.scatter(X_tr[:, 0] * X_scale[0] + X_centr[0], X_tr[:, 1] * X_scale[1] +
                X_centr[1], c="black", s=10, alpha=0.5, marker="s")  # different shape
    plt.scatter(X_te[:, 0] * X_scale[0] + X_centr[0], X_te[:, 1] * X_scale[1] + X_centr[1], c=nll_bma_each,
                s=abs(nll_bma_each)*200, cmap='inferno_r', norm=color_norm_nll_each)

    cbar = plt.colorbar()

    plt.title("Fold " + str(fold_num))

    plt.xlim(min(coordinate[:, 0]), max(coordinate[:, 0]))
    plt.ylim(min(coordinate[:, 1]), max(coordinate[:, 1]))
    plt.show()

    # plot the bae_bma_diff_df
    plt.figure(figsize=(5, 4))
    plt.scatter(X_tr[:, 0] * X_scale[0] + X_centr[0], X_tr[:, 1] * X_scale[1] +
            X_centr[1], c="black", s=10, alpha=0.5, marker="s")  # different shape
    plt.scatter(X_te[:, 0] * X_scale[0] + X_centr[0], X_te[:, 1] * X_scale[1] + X_centr[1], c=nll_bae_each,
                s=abs(nll_bae_each)*200, cmap='inferno_r', norm=color_norm_nll_each)

    cbar = plt.colorbar()

    plt.title("Fold " + str(fold_num))

    plt.xlim(min(coordinate[:, 0]), max(coordinate[:, 0]))
    plt.ylim(min(coordinate[:, 1]), max(coordinate[:, 1]))
    plt.show()  

#print(bae_bma_diff_line)
